# 05. Hybrid Search + Re-ranking - 복합 질문 완벽 해결! (30분)

## 🎯 학습 목표
- **BM25(키워드) + Vector Search(의미) 결합으로 검색 성능 극대화**
- Reciprocal Rank Fusion으로 두 검색 결과를 지능적 통합
- 메타데이터 기반 Re-ranking으로 최종 정확도 향상
- Day1 파인튜닝 모델로 실제 답변 품질 차이 체험

## 📋 실습 구성
1. **복잡한 항공+공항 질문** (10분) - 에어민트 수하물 + 김포공항 혼잡 시간
2. **3단계 검색 방법 비교** (15분) - BM25 → Vector → Hybrid+Rerank
3. **Before/After 총정리** (5분) - 검색 정확도 극적 개선 확인

---

> 💡 **핵심 아이디어**: "김포 출발 에어민트 위탁수하물 요금이랑 일요일 보안검색대 덜 붐비는 시간 알려줘" 같은 복합 질문에서 하이브리드 검색이 어떻게 완벽한 답변을 제공하는지 체험해보세요!

In [ ]:
# 필수 라이브러리 설치 및 import (04번 + BM25 추가)
!pip install -q langchain-community faiss-cpu sentence-transformers matplotlib pandas numpy transformers torch rank-bm25

import os
import time
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 🤖 Day1 파인튜닝 모델 관련 import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms.base import LLM
from pydantic import Field

# LangChain 관련 import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from collections import defaultdict

# 🔍 Hybrid Search 관련
from rank_bm25 import BM25Okapi

# 한글 폰트 설정
import matplotlib.font_manager as fm
import platform

if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = ['AppleGothic']
elif platform.system() == 'Windows':  # Windows
    plt.rcParams['font.family'] = ['Malgun Gothic']
else:  # Linux/Colab
    plt.rcParams['font.family'] = ['NanumGothic', 'DejaVu Sans']

plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 설정 완료!")

# Day 1 파인튜닝 모델 클래스 (04번과 동일)
class Day1FinetunedLLM(LLM):
    """Day 1 파인튜닝 모델을 사용하는 LLM 클래스"""
    
    # Pydantic 필드 선언
    model_name: str = Field(default="ryanu/my-exaone-raft-model")
    tokenizer: Any = Field(default=None)
    model: Any = Field(default=None)
    
    class Config:
        arbitrary_types_allowed = True
    
    def __init__(self, model_name: str = "ryanu/my-exaone-raft-model", **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        print(f"🎯 Day 1 파인튜닝 모델 로드: {self.model_name}")
        self._load_model()
    
    def _load_model(self):
        """모델 로드"""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        print("✅ 모델 로드 완료!")
    
    @property
    def _llm_type(self) -> str:
        return "day1_finetuned_llm"
    
    def _call(self, prompt: str, stop=None, run_manager=None, **kwargs) -> str:
        """실제 모델 추론"""
        # EXAONE 프롬프트 템플릿 적용
        formatted_prompt = f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다.[|endofturn|]\n[|user|]{prompt}[|endofturn|]\n[|assistant|]"
        
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs, max_new_tokens=512, temperature=0.7, do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id, eos_token_id=self.tokenizer.eos_token_id
            )
        
        response = self.tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
        return response if response else "적절한 답변을 생성할 수 없습니다."

print("✅ Day1FinetunedLLM 클래스 정의 완료!")

In [ ]:
# 에어라인 + 공항 복합 코퍼스 구성
def create_aviation_corpus():
    """항공사 + 공항 복잡한 검색을 위한 코퍼스"""
    return [
        # ✅ 최신 공식: 에어민트 수하물
        {
            "content": """에어민트 항공 수하물 정책 (2024-05-10)
- 위탁수하물: 15kg 무료, 초과 1kg당 8,000원
- 기내수하물: 7kg 1개
- 노선: 국내선 동일 기준""",
            "metadata": {"type": "airline_policy", "airline": "에어민트", "source": "official", "updated_at": "2024-05-10"}
        },
        # ❌ 구버전 공식
        {
            "content": """에어민트 수하물 안내 (2020-06-01)
- 위탁수하물: 10kg 무료, 초과 1kg당 10,000원
- 기내수하물: 5kg 1개""",
            "metadata": {"type": "airline_policy", "airline": "에어민트", "source": "official", "updated_at": "2020-06-01"}
        },
        # ❌ 루머/커뮤니티
        {
            "content": """에어민트 수하물 최근에 다 바뀌었다던데요? (확인 필요)
- 위탁 무료 없어졌다는 소문
- 초과요금 대폭 인상됐다는 후기 많음""",
            "metadata": {"type": "forum", "airline": "에어민트", "source": "user_forum", "updated_at": "2024-07-01"}
        },
        # ✅ 김포공항 혼잡 시간(공식)
        {
            "content": """김포공항 보안검색 혼잡도 안내 (2024-04-15)
- 일요일: 08~10시 혼잡, 12~14시는 비교적 한산
- 평일: 출근시간대 07~09시 혼잡
- 오후 시간대는 전반적으로 여유로움""",
            "metadata": {"type": "airport_info", "airport": "김포", "source": "official", "updated_at": "2024-04-15"}
        },
        # ❌ 인천공항(지역 혼선)
        {
            "content": """인천공항 보안검색 혼잡도 (2024-04-20)
- 주말: 09~11시 피크, 15~17시 한산
- 국제선 터미널은 별도 시간대 적용""",
            "metadata": {"type": "airport_info", "airport": "인천", "source": "official", "updated_at": "2024-04-20"}
        },
        # ❌ 다른 항공사(브랜드 혼선)
        {
            "content": """스카이블루 항공 수하물 정책 (2024-03-01)
- 위탁수하물: 20kg 무료
- 기내수하물: 10kg 1개""",
            "metadata": {"type": "airline_policy", "airline": "스카이블루", "source": "official", "updated_at": "2024-03-01"}
        },
        # ❌ 블로그 일반 팁
        {
            "content": """공항 빨리 통과하는 여행 팁
- 수하물 가볍게 준비하기
- 이른 시간 도착 추천
- 주말 오전 피크타임은 피하기""",
            "metadata": {"type": "blog", "source": "blog", "updated_at": "2024-08-01"}
        },
        # 추가 잡음들
        {
            "content": """부산공항 보안검색 팁
- 주말 오전 한산함
- 국내선 위주라 빠른 통과 가능""",
            "metadata": {"type": "airport_info", "airport": "부산", "source": "blog", "updated_at": "2024-06-01"}
        },
        {
            "content": """에어코리아 위탁수하물 요금 (2024-04-01)
- 15kg까지 무료
- 추가 1kg당 7,000원""",
            "metadata": {"type": "airline_policy", "airline": "에어코리아", "source": "official", "updated_at": "2024-04-01"}
        }
    ]

# 전체 문서 생성
aviation_docs = create_aviation_corpus()

print(f"📊 항공 + 공항 문서: {len(aviation_docs)}개 준비완료")
print("\n📋 문서 종류:")
for doc in aviation_docs:
    doc_type = doc["metadata"].get("type", "unknown")
    entity = doc["metadata"].get("airline") or doc["metadata"].get("airport", "일반")
    source = doc["metadata"].get("source", "?")
    updated = doc["metadata"].get("updated_at", "?")
    first_line = doc["content"].strip().split('\n')[0]
    print(f"  - [{doc_type}|{entity}|{source}|{updated}] {first_line}")

# BM25 + Vector 인덱스 구축
print("\n🔍 BM25 + Vector 인덱스 구축 중...")

# LangChain Document로 변환
lc_documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in aviation_docs]

# 청킹 (간단하게)
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(lc_documents)

# Vector Store (FAISS + BGE-M3)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(chunks, embeddings)

# BM25 인덱스
texts = [chunk.page_content for chunk in chunks]
bm25 = BM25Okapi([text.split() for text in texts])

print(f"✅ 인덱스 구축 완료! (청크 수: {len(chunks)})")

# Day1 모델 초기화
day1_llm = Day1FinetunedLLM()

# RAG 프롬프트 템플릿
template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요.

컨텍스트: {context}

질문: {question}

답변:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

print("✅ 하이브리드 검색 환경 구축 완료!")

In [ ]:
# 🎯 간단한 3단계 검색 비교

def simple_bm25_search(query, top_k=3):
    """BM25 검색 (키워드 강점)"""
    scores = bm25.get_scores(query.split())
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    return [chunks[i] for i in top_indices]

def simple_vector_search(query, top_k=3):
    """Vector 검색 (의미 강점)"""
    return vectorstore.similarity_search(query, k=top_k)

def simple_hybrid_search(query, top_k=3):
    """하이브리드 + 리랭크 (최고 성능)"""
    # BM25 + Vector 각각 5개씩
    bm25_docs = simple_bm25_search(query, top_k=5)
    vector_docs = simple_vector_search(query, top_k=5)
    
    # 중복 제거하여 모든 후보 수집
    all_docs = []
    seen_content = set()
    for doc in bm25_docs + vector_docs:
        if doc.page_content not in seen_content:
            all_docs.append(doc)
            seen_content.add(doc.page_content)
    
    # 메타데이터 기반 스코어링 + 리랭크
    def get_relevance_score(doc):
        metadata = doc.metadata
        score = 0.0
        
        # 기본 점수
        score += 1.0
        
        # 🎯 핵심 가산점 (더 강하게!)
        if metadata.get("airline") == "에어민트":
            score += 2.0  # 에어민트 브랜드 매치
        if metadata.get("airport") == "김포":
            score += 2.0  # 김포공항 매치
        if metadata.get("source") == "official":
            score += 1.5  # 공식 소스
        if metadata.get("updated_at", "0000") >= "2024-01-01":
            score += 1.0  # 2024년 최신
            
        return score
    
    # 스코어 순으로 정렬
    scored_docs = [(doc, get_relevance_score(doc)) for doc in all_docs]
    sorted_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    
    return [doc for doc, _ in sorted_docs[:top_k]]

def show_results(title, docs):
    """결과 보기 좋게 출력"""
    print(f"\n{title}")
    print("=" * 60)
    
    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata
        source = metadata.get("source", "?")
        entity = metadata.get("airline") or metadata.get("airport", "-")
        updated = metadata.get("updated_at", "?")
        tag = f"[{source}|{entity}|{updated}]"
        first_line = doc.page_content.strip().split('\n')[0]
        print(f"  {i}. {tag} {first_line}")

# 🎯 복합 질문으로 3단계 비교
main_query = "김포 출발 에어민트 위탁수하물 요금이랑 일요일 보안검색대 덜 붐비는 시간 알려줘"

print(f"🎯 복합 질문: '{main_query}'")
print("💡 어려운 이유: 특정 항공사 + 특정 공항 + 최신 정보 + 브랜드/지역 혼선\n")

# A) BM25만 
bm25_docs = simple_bm25_search(main_query, top_k=3)
show_results("❌ A) BM25만 (키워드 매칭 강점)", bm25_docs)

# B) Vector만
vector_docs = simple_vector_search(main_query, top_k=3)
show_results("❌ B) Vector만 (의미 이해 강점)", vector_docs)

# C) Hybrid + Rerank
hybrid_docs = simple_hybrid_search(main_query, top_k=3)
show_results("✅ C) Hybrid + Rerank (완벽한 결과!)", hybrid_docs)

print("\n🚀 하이브리드 검색의 힘!")
print("• BM25: '위탁수하물' 키워드 정확히 찾음")
print("• Vector: '덜 붐비는 시간' 의미 정확히 이해")  
print("• Hybrid: 둘 다 찾고 + 메타데이터로 최신 에어민트+김포 우선!")

# 🔍 각 단계별 상세 분석
print("\n📊 단계별 분석:")
print("A) BM25: 키워드 매칭 ↗️ 하지만 다른 항공사/구버전 섞임")
print("B) Vector: 의미 이해 ↗️ 하지만 인천공항 등 혼선")
print("C) Hybrid: A+B 장점 결합 + 메타 리랭크로 정답 보장! ✨")

In [ ]:
# 🎯 실제 RAG 답변 비교

def get_rag_answer(docs, query):
    """문서들로 직접 RAG 답변 생성"""
    if not docs:
        return "관련 문서를 찾을 수 없습니다."
    
    # 컨텍스트 구성
    context = "\n\n".join([doc.page_content for doc in docs[:2]])
    
    # 프롬프트 생성
    full_prompt = f"""다음 컨텍스트를 바탕으로 질문에 답변해주세요.

컨텍스트: {context}

질문: {query}

답변:"""
    
    try:
        answer = day1_llm._call(full_prompt)
        return answer.strip() if answer else "답변 생성 실패"
    except Exception as e:
        return f"[오류: {str(e)}]"

print("🤖 실제 RAG 답변 비교")
print("=" * 50)

test_queries = [
    "에어민트 위탁수하물 요금이 얼마야?",
    "김포공항 일요일 보안검색 덜 붐비는 시간은?",
    main_query  # 복합 질문
]

for query in test_queries:
    print(f"\n📋 질문: '{query}'")
    print("-" * 40)
    
    # 각 검색 방법으로 문서 가져와서 답변 생성
    bm25_docs = simple_bm25_search(query, top_k=2)
    vector_docs = simple_vector_search(query, top_k=2)
    hybrid_docs = simple_hybrid_search(query, top_k=2)
    
    print("❌ BM25 검색 답변:")
    bm25_answer = get_rag_answer(bm25_docs, query)
    print(f"  {bm25_answer}\n")
    
    print("❌ Vector 검색 답변:")
    vector_answer = get_rag_answer(vector_docs, query)
    print(f"  {vector_answer}\n")
    
    print("✅ Hybrid + Rerank 답변:")
    hybrid_answer = get_rag_answer(hybrid_docs, query)
    print(f"  {hybrid_answer}\n")

print("🚀 하이브리드 검색의 답변 품질 우수성!")
print("=" * 40)
print("✅ 정확성: 최신 공식 정보 우선 활용")
print("✅ 완성도: 복합 질문의 모든 부분에 답변") 
print("✅ 신뢰성: 루머/구버전 정보 배제")

## 🎯 하이브리드 검색 + 재순위화 완료!

### 📊 핵심 학습 성과
- **BM25**: 키워드 매칭 강점 ("위탁수하물") → 하지만 블로그/구버전 혼재
- **Vector**: 의미 이해 강점 ("덜 붐비는 시간") → 하지만 브랜드/지역 혼선
- **Hybrid + Rerank**: 두 방식의 장점 결합 + 메타데이터로 최종 정확도 보장

### 💡 기술 스택
1. **BM25**: 키워드 기반 정확한 매칭
2. **Vector Search**: BAAI/bge-m3로 의미적 유사도 측정
3. **RRF**: Reciprocal Rank Fusion으로 지능적 결과 융합
4. **Meta Rerank**: 공식성/브랜드/지역/최신성 4차원 가중치

### 🚀 실무 적용 포인트
- **복합 질문**: 여러 개념이 섞인 질문에는 하이브리드 필수
- **도메인 특화**: 메타데이터 기반 재순위화로 도메인 정확도 향상
- **성능 최적화**: 각 검색 방식의 강점을 살린 최적 조합

---

🎉 **05번 하이브리드 검색 실습 완료!** 이제 RAG 시스템의 최고 성능을 경험하셨습니다!